# Capstone Project
## Neural translation model
### Instructions

In this notebook, you will create a neural network that translates from English to German. You will use concepts from throughout this course, including building more flexible model architectures, freezing layers, data processing pipeline and sequence modelling.

This project is peer-assessed. Within this notebook you will find instructions in each section for how to complete the project. Pay close attention to the instructions as the peer review will be carried out according to a grading rubric that checks key parts of the project instructions. Feel free to add extra cells into the notebook as required.

### How to submit

When you have completed the Capstone project notebook, you will submit a pdf of the notebook for peer review. First ensure that the notebook has been fully executed from beginning to end, and all of the cell outputs are visible. This is important, as the grading rubric depends on the reviewer being able to view the outputs of your notebook. Save the notebook as a pdf (File -> Download as -> PDF via LaTeX). You should then submit this pdf for review.

### Let's get started!

We'll start by running some imports, and loading the dataset. For this project you are free to make further imports throughout the notebook as you wish. 

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import unicodedata
import re
import json 
import tensorflow_hub as hub 
import numpy as np 
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Layer, Dense, Conv2D, MaxPooling2D, Flatten, Masking, LSTM

![Flags overview image](data/germany_uk_flags.png)

For the capstone project, you will use a language dataset from http://www.manythings.org/anki/ to build a neural translation model. This dataset consists of over 200,000 pairs of sentences in English and German. In order to make the training quicker, we will restrict to our dataset to 20,000 pairs. Feel free to change this if you wish - the size of the dataset used is not part of the grading rubric.

Your goal is to develop a neural translation model from English to German, making use of a pre-trained English word embedding module.

In [2]:
# Run this cell to load the dataset

NUM_EXAMPLES = 20000
data_examples = []
with open('data/deu.txt', 'r', encoding='utf8') as f:
    for line in f.readlines():
        if len(data_examples) < NUM_EXAMPLES:
            data_examples.append(line)
        else:
            break

print(data_examples)

['Hi.\tHallo!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)\n', 'Hi.\tGrüß Gott!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)\n', 'Run!\tLauf!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #941078 (Fingerhut)\n', 'Wow!\tPotzdonner!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #2122382 (Pfirsichbaeumchen)\n', 'Wow!\tDonnerwetter!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #2122391 (Pfirsichbaeumchen)\n', 'Fire!\tFeuer!\tCC-BY 2.0 (France) Attribution: tatoeba.org #1829639 (Spamster) & #1958697 (Tamy)\n', 'Help!\tHilfe!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #575889 (MUIRIEL)\n', 'Help!\tZu Hülf!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #2122375 (Pfirsichbaeumchen)\n', 'Stop!\tStopp!\tCC-BY 2.0 (France) Attribution: tatoeba.org #448320 (FeuDRenais) & #626467 (jakov)\n', 'Wait!\tWarte!\tCC-BY 2.0 (France) Attri

In [3]:
# These functions preprocess English and German sentences

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"ü", 'ue', sentence)
    sentence = re.sub(r"ä", 'ae', sentence)
    sentence = re.sub(r"ö", 'oe', sentence)
    sentence = re.sub(r'ß', 'ss', sentence)
    
    sentence = unicode_to_ascii(sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r"[^a-z?.!,']+", " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    return sentence.strip()

#### The custom translation model
The following is a schematic of the custom translation model architecture you will develop in this project.

![Model Schematic](data/neural_translation_model.png)

Key:
![Model key](data/neural_translation_model_key.png)

The custom model consists of an encoder RNN and a decoder RNN. The encoder takes words of an English sentence as input, and uses a pre-trained word embedding to embed the words into a 128-dimensional space. To indicate the end of the input sentence, a special end token (in the same 128-dimensional space) is passed in as an input. This token is a TensorFlow Variable that is learned in the training phase (unlike the pre-trained word embedding, which is frozen).

The decoder RNN takes the internal state of the encoder network as its initial state. A start token is passed in as the first input, which is embedded using a learned German word embedding. The decoder RNN then makes a prediction for the next German word, which during inference is then passed in as the following input, and this process is repeated until the special `<end>` token is emitted from the decoder.

## 1. Text preprocessing
* Create separate lists of English and German sentences, and preprocess them using the `preprocess_sentence` function provided for you above.
* Add a special `"<start>"` and `"<end>"` token to the beginning and end of every German sentence.
* Use the Tokenizer class from the `tf.keras.preprocessing.text` module to tokenize the German sentences, ensuring that no character filters are applied. _Hint: use the Tokenizer's "filter" keyword argument._
* Print out at least 5 randomly chosen examples of (preprocessed) English and German sentence pairs. For the German sentence, print out the text (with start and end tokens) as well as the tokenized sequence.
* Pad the end of the tokenized German sequences with zeros, and batch the complete set of sequences into one numpy array.

In [4]:
english_sentences = []
german_sentences = []

for i in range(len(data_examples)):
    english_sentences.append((preprocess_sentence(re.split('\t', data_examples[i])[0])))
    german_sentences.append('<start>' + preprocess_sentence(re.split('\t', data_examples[i])[0]))

In [5]:
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(german_sentences)
tokenizer_config = tokenizer.get_config()
word_index = json.loads(tokenizer_config['word_index'])
index_words = json.loads(tokenizer_config['index_word'])

In [6]:
print(word_index.get("."))
print(word_index.get(","))

1
38


In [7]:
german_sentences_seq = tokenizer.texts_to_sequences(german_sentences)

In [8]:
num_samples = 5
random_ind = np.random.choice(len(english_sentences), num_samples, replace=False)
print(random_ind)
for i, (chunk1, chunk2) in enumerate(zip(np.array(english_sentences)[random_ind], np.array(german_sentences)[random_ind])):
    print(i, chunk1)
    print(i, chunk1, chunk2)

[ 7761  7590 17120 11136  2093]
0 it was perfect .
0 it was perfect . <start>it was perfect .
1 i'm interested .
1 i'm interested . <start>i'm interested .
2 tom was captured .
2 tom was captured . <start>tom was captured .
3 it was painless .
3 it was painless . <start>it was painless .
4 i want kids .
4 i want kids . <start>i want kids .


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
german_sentences_padded = pad_sequences(german_sentences_seq, maxlen=None,
                                       padding='post',
                                       value=0)

## 2. Prepare the data with tf.data.Dataset objects

#### Load the embedding layer
As part of the dataset preproceessing for this project, you will use a pre-trained English word embedding module from TensorFlow Hub. The URL for the module is https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1. This module has also been made available as a complete saved model in the folder `'./models/tf2-preview_nnlm-en-dim128_1'`. 

This embedding takes a batch of text tokens in a 1-D tensor of strings as input. It then embeds the separate tokens into a 128-dimensional space. 

The code to load and test the embedding layer is provided for you below.

**NB:** this model can also be used as a sentence embedding module. The module will process each token by removing punctuation and splitting on spaces. It then averages the word embeddings over a sentence to give a single embedding vector. However, we will use it only as a word embedding module, and will pass each word in the input sentence as a separate token.

In [ ]:
# Load embedding module from Tensorflow Hub

embedding_layer = tf.keras.models.load_model('./models/tf2-preview_nnlm-en-dim128_1')

In [ ]:
# Test the layer

embedding_layer(tf.constant(["these", "aren't", "the", "droids", "you're", "looking", "for"])).shape

TensorShape([7, 128])

You should now prepare the training and validation Datasets.

* Create a random training and validation set split of the data, reserving e.g. 20% of the data for validation (NB: each English dataset example is a single sentence string, and each German dataset example is a sequence of padded integer tokens).
* Load the training and validation sets into a tf.data.Dataset object, passing in a tuple of English and German data for both training and validation sets.
* Create a function to map over the datasets that splits each English sentence at spaces. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.strings.split function._
* Create a function to map over the datasets that embeds each sequence of English words using the loaded embedding layer/model. Apply this function to both Dataset objects using the map method.
* Create a function to filter out dataset examples where the English sentence is more than 13 (embedded) tokens in length. Apply this function to both Dataset objects using the filter method.
* Create a function to map over the datasets that pads each English sequence of embeddings with some distinct padding value before the sequence, so that each sequence is length 13. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.pad function. You can extract a Tensor shape using tf.shape; you might also find the tf.math.maximum function useful._
* Batch both training and validation Datasets with a batch size of 16.
* Print the `element_spec` property for the training and validation Datasets. 
* Using the Dataset `.take(1)` method, print the shape of the English data example from the training Dataset.
* Using the Dataset `.take(1)` method, print the German data example Tensor from the validation Dataset.

In [ ]:
x_train_english, x_test_english, x_train_german_tokenised, x_test_german_tokenised = train_test_split(english_sentences, 
                                                                                                     german_sentences_padded, 
                                                                                                     test_size=0.2,
                                                                                                     )
training_dataset = tf.data.Dataset.from_tensor_slices((x_train_english, x_train_german_tokenised))
validation_dataset = tf.data.Dataset.from_tensor_slices((x_test_english, x_test_german_tokenised))
print(training_dataset.element_spec)

(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(7,), dtype=tf.int32, name=None))


In [ ]:
def split_space(dataset):
    def split(english, german):
        return tf.strings.split(english), german
    
    return dataset.map(split)

train_data = split_space(training_dataset)
valid_data = split_space(validation_dataset)

print(train_data.element_spec)
print(valid_data.element_spec)

(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(7,), dtype=tf.int32, name=None))
(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(7,), dtype=tf.int32, name=None))


In [ ]:
def embed_english(dataset):
    def embed(english, german):
        return embedding_layer(english), german
    return dataset.map(embed)

train_embed_data = embed_english(train_data)
valid_embed_data = embed_english(valid_data)

print(train_embed_data.element_spec)
print(valid_embed_data.element_spec)

(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(7,), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(7,), dtype=tf.int32, name=None))


In [ ]:
def filter_len(dataset):
    def chop_len(english, german):
        return len(english) < 14
    
    return dataset.filter(chop_len)

train_len_data = filter_len(train_embed_data)
valid_len_data = filter_len(valid_embed_data)

print(train_len_data.element_spec)
print(valid_len_data.element_spec)

(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(7,), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(7,), dtype=tf.int32, name=None))


In [ ]:
def pad_data(dataset):
    def pad_elements(english, german):
        return tf.pad(english, [[13 - len(english), 0],[0, 0]], "CONSTANT", constant_values=0), german
    return dataset.map(pad_elements)

train_pad_data = pad_data(train_len_data)
valid_pad_data = pad_data(valid_len_data)

train_batch_data = train_pad_data.batch(16)
valid_batch_data = valid_pad_data.batch(16)

print(train_batch_data.element_spec)
print(valid_batch_data.element_spec)

(TensorSpec(shape=(None, None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7), dtype=tf.int32, name=None))
(TensorSpec(shape=(None, None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7), dtype=tf.int32, name=None))


## 3. Create the custom layer
You will now create a custom layer to add the learned end token embedding to the encoder model:

![Encoder schematic](data/neural_translation_model_encoder.png)

You should now build the custom layer.
* Using layer subclassing, create a custom layer that takes a batch of English data examples from one of the Datasets, and adds a learned embedded ‘end’ token to the end of each sequence. 
* This layer should create a TensorFlow Variable (that will be learned during training) that is 128-dimensional (the size of the embedding space). _Hint: you may find it helpful in the call method to use the tf.tile function to replicate the end token embedding across every element in the batch._
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and print the shape. Test the custom layer by calling the layer on the English data batch Tensor and print the resulting Tensor shape (the layer should increase the sequence length by one).

In [ ]:
class CustomLayer(Layer):
    
    def __init__(self):
        super().__init__()
        self.W = tf.random_normal_initializer()
        self.W = tf.Variable(initial_value=self.W(shape=(1, 1, 128)), trainable=True)
        
    def call(self, inputs):
        multiple = tf.constant([inputs.shape[0], 1, 1], tf.int32)
        embeddings = tf.tile(self.W, multiple)
        return tf.keras.layers.concatenate([inputs, embeddings], axis=1)

In [ ]:
for english, german in train_batch_data.take(1):
    custom_layer = CustomLayer()
    output = custom_layer(english)
    print(output.shape)

## 4. Build the encoder network
The encoder network follows the schematic diagram above. You should now build the RNN encoder model.
* Using the functional API, build the encoder network according to the following spec:
    * The model will take a batch of sequences of embedded English words as input, as given by the Dataset objects.
    * The next layer in the encoder will be the custom layer you created previously, to add a learned end token embedding to the end of the English sequence.
    * This is followed by a Masking layer, with the `mask_value` set to the distinct padding value you used when you padded the English sequences with the Dataset preprocessing above.
    * The final layer is an LSTM layer with 512 units, which also returns the hidden and cell states.
    * The encoder is a multi-output model. There should be two output Tensors of this model: the hidden state and cell states of the LSTM layer. The output of the LSTM layer is unused.
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and test the encoder model by calling it on the English data Tensor, and print the shape of the resulting Tensor outputs.
* Print the model summary for the encoder network.

In [ ]:
def encoder(input_shape, batch_size):
    inputs = Input(shape=input_shape, batch_size=batch_size)
    x = CustomLayer()(inputs)
    x = Masking(mask_value=0)(x)
    _, hidden_layer, cell_state = LSTM(512, return_sequences=True, return_state=True)(x)
    model = Model(inputs=inputs, outputs=[_, hidden_layer, cell_state])
    return model

In [ ]:
for english, german in train_batch_data.take(1):
    encoder = encoder(english.shape[1:], english.shape[0])
    out, hidden, cell_state = encoder(english)
    print(f"out shape: {out.shape}, hidden state shape: {hidden.shape}, cell state shape: {cell_state.shape}")

In [ ]:
encoder.summary()

## 5. Build the decoder network
The decoder network follows the schematic diagram below. 

![Decoder schematic](data/neural_translation_model_decoder.png)

You should now build the RNN decoder model.
* Using Model subclassing, build the decoder network according to the following spec:
    * The initializer should create the following layers:
        * An Embedding layer with vocabulary size set to the number of unique German tokens, embedding dimension 128, and set to mask zero values in the input.
        * An LSTM layer with 512 units, that returns its hidden and cell states, and also returns sequences.
        * A Dense layer with number of units equal to the number of unique German tokens, and no activation function.
    * The call method should include the usual `inputs` argument, as well as the additional keyword arguments `hidden_state` and `cell_state`. The default value for these keyword arguments should be `None`.
    * The call method should pass the inputs through the Embedding layer, and then through the LSTM layer. If the `hidden_state` and `cell_state` arguments are provided, these should be used for the initial state of the LSTM layer. _Hint: use the_ `initial_state` _keyword argument when calling the LSTM layer on its input._
    * The call method should pass the LSTM output sequence through the Dense layer, and return the resulting Tensor, along with the hidden and cell states of the LSTM layer.
* Using the Dataset `.take(1)` method, extract a batch of English and German data examples from the training Dataset. Test the decoder model by first calling the encoder model on the English data Tensor to get the hidden and cell states, and then call the decoder model on the German data Tensor and hidden and cell states, and print the shape of the resulting decoder Tensor outputs.
* Print the model summary for the decoder network.

In [ ]:
from tensorflow.keras.layers import Embedding

class decoder(Model):
    
    def __init__(self):
        super().__init__()
        self.embed = Embedding(input_dim=max(tokenizer.word_index.values()) + 1,
                                            output_dim=128,
                                            mask_zero=True)
        self.lstm = LSTM(512, return_sequences=True, return_state=True)
        self.dense = Dense(max(tokenizer.word_index.values()) + 1)
        
    def call(self, inputs, hidden_state=None, cell_state=None):
        x = self.embed(inputs)
        output_seq, output_hidden_state, output_cell_state = self.lstm(x, initial_state=[hidden_state, cell_state])
        x = self.dense(output_seq)
        return x, output_hidden_state, output_cell_state


In [ ]:
decoder = decoder()
for english, german in train_batch_data.take(1):
    _, encoder_hidden, encoder_cell_state = encoder(english)
    x , decoder_hidden, decoder_cell = decoder(german, encoder_hidden, encoder_cell_state)
    print(f"x.shape: {x.shape}, hidden_state_shape: {decoder_hidden.shape}, cell_state_shape: {decoder_cell.shape}")

In [ ]:
print(decoder.summary())

## 6. Make a custom training loop
You should now write a custom training loop to train your custom neural translation model.
* Define a function that takes a Tensor batch of German data (as extracted from the training Dataset), and returns a tuple containing German inputs and outputs for the decoder model (refer to schematic diagram above).
* Define a function that computes the forward and backward pass for your translation model. This function should take an English input, German input and German output as arguments, and should do the following:
    * Pass the English input into the encoder, to get the hidden and cell states of the encoder LSTM.
    * These hidden and cell states are then passed into the decoder, along with the German inputs, which returns a sequence of outputs (the hidden and cell state outputs of the decoder LSTM are unused in this function).
    * The loss should then be computed between the decoder outputs and the German output function argument.
    * The function returns the loss and gradients with respect to the encoder and decoder’s trainable variables.
    * Decorate the function with @tf.function
* Define and run a custom training loop for a number of epochs (for you to choose) that does the following:
    * Iterates through the training dataset, and creates decoder inputs and outputs from the German sequences.
    * Updates the parameters of the translation model using the gradients of the function above and an optimizer object.
    * Every epoch, compute the validation loss on a number of batches from the validation and save the epoch training and validation losses.
* Plot the learning curves for loss vs epoch for both training and validation sets.

_Hint: This model is computationally demanding to train. The quality of the model or length of training is not a factor in the grading rubric. However, to obtain a better model we recommend using the GPU accelerator hardware on Colab._

In [ ]:
def decoder_inputs_outputs(batch):
    german_inputs = batch[:, :-1]
    german_outputs = batch[:, 1:]
    return german_inputs, german_outputs

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
@tf.function
def forward_backward(encoder, decoder, english_inputs, german_inputs, german_outputs, loss):
    with tf.GradientTape() as tape:
        _, hidden_state, cell_state = encoder(english_inputs)
        decoder_outputs, _, _ = decoder(german_inputs, hidden_state, cell_state)
        loss_val = loss(german_outputs, decoder_outputs)
        grads = tape.gradient(loss_val, encoder.trainable_variables + decoder.trainable_variables)
        return (loss_val, grads)

In [ ]:
epoch_train_loss = []
epoch_valid_loss = []
num_epochs = 10
val_steps = 10 
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

In [ ]:
for epoch in range(num_epochs):
    
    train_epoch_loss_avg = tf.keras.metrics.Mean()
    val_epoch_loss_avg = tf.keras.metrics.Mean()
    
    for english, german in train_batch_data:
        german_inputs, german_outputs = decoder_inputs_outputs(german)
        train_loss, grads = forward_backward(encoder, decoder, english, german_inputs, german_outputs, loss)
        optimizer.apply_gradients(zip(grads, encoder.trainable_variables + decoder.trainable_variables))
        train_epoch_loss_avg(train_loss)
    
    for english, german in valid_batch_data.take(val_steps):
        german_inputs, german_outputs = decoder_inputs_outputs(german)
        valid_loss, _ = forward_backward(encoder, decoder, english, german_inputs, german_outputs, loss)
        val_epoch_loss_avg(valid_loss)
    
    epoch_train_loss.append(train_epoch_loss_avg.result().numpy())
    epoch_valid_loss.append(val_epoch_loss_avg.result().numpy())
    print(f"epochs: {epoch + 1} train loss: {train_epoch_loss_avg.result().numpy()} valid loss: {val_epoch_loss_avg.result().numpy()}")

In [ ]:
plt.figure(figsize = (10,8))

plt.plot(epoch_train_loss, "-")
plt.plot(epoch_valid_loss, "--")
plt.title("Loss Vs Epoch")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Training", "Validation"], loc ="best")
plt.show()

## 7. Use the model to translate
Now it's time to put your model into practice! You should run your translation for five randomly sampled English sentences from the dataset. For each sentence, the process is as follows:
* Preprocess and embed the English sentence according to the model requirements.
* Pass the embedded sentence through the encoder to get the encoder hidden and cell states.
* Starting with the special  `"<start>"` token, use this token and the final encoder hidden and cell states to get the one-step prediction from the decoder, as well as the decoder’s updated hidden and cell states.
* Create a loop to get the next step prediction and updated hidden and cell states from the decoder, using the most recent hidden and cell states. Terminate the loop when the `"<end>"` token is emitted, or when the sentence has reached a maximum length.
* Decode the output token sequence into German text and print the English text and the model's German translation.

In [ ]:
random_sample = np.random.choice(range(len(data_examples)), 5, replace=False)

In [ ]:
for index in random_sample.tolist():
    
    output_seq = []
    preprocessed_sequence = preprocess_sentence(Eng_sentences[index])
    split_sequence = tf.string.split(preprocessed_sequence)
    
    if len(split_sequence) > 13:
        split_sequence = split_sequence[:13]
    
    embed_sentence = embedding_layer(split_sentence)
    padded_sentence = tf.pad(embed_sentence, [[13 - len(embed_sentence), 0], [0, 0]], 'CONSTANT', constant_values = 0)

    _, encoder_hidden_state, encoder_cell_state = encoder(tf.expand_dims(padded_sentence, axis = 0))

    start_token = tf.reshape(tokenizer.word_index["<start>"], [1,-1])
    decoder_op ,decoder_hidden_state, decoder_cell_state  = decoder(start_token, encoder_hidden_state, encoder_cell_state)
    predicted_index = np.argmax(decoder_op)
    output_sequence.append(predicted_index)

    while (tokenizer.index_word[predicted_index] != "<end>") and (len(output_sequence) < 20):
        decoder_op ,decoder_hidden_state, decoder_cell_state  = decoder(tf.reshape(predicted_index, [1,-1]), decoder_hidden_state, decoder_cell_state)
        predicted_index = np.argmax(decoder_op)
        output_sequence.append(predicted_index)

    predicted_words = [tokenizer.index_word[token] for token in output_sequence]
    predicted_german = ' '.join(predicted_words)
    print("English text:{}\nGerman translation :{}".format(Eng_sentences[index], predicted_german))